# Mergear shps para crear las capas de CartoDB

CartoDB permite el uso de sólo 4 capas para sus usuarios básicos, de manera que se mergearán los shps creando 4 capas principales: 
    1. Divisiones políticas de la CABA y sus indicadores (radios, fracciones, barrios y comunas)
    2. Buffers de los points y sus indicadores.
    3. Estaciones de subte, metrobús y ferrocarril.
    4. Líneas de subte, metrobús, ferrocarril y colectivos.

In [44]:
import os
import scripts.path_finders as pf
from scripts.cartodb_layers import merge_shapefiles
import shapefile
from scripts.create_indicators import get_or_create_indicators_df
from scripts import recalculate_indicators

## 1. Divisiones políticas de la CABA y sus indicadores (radios, fracciones, barrios y comunas)

In [45]:
paths_1 = {
    "RADIO": pf.get_indic("RADIO", "shp"),
    "BARRIO": pf.get_indic("BARRIO", "shp"),
    "DPTO": pf.get_indic("comunas_censo_2010"),
    "FRAC": pf.get_indic("fracciones_censo_2010")
}
group_fields_by_sf = {
    "RADIO": {"co_frac_ra": "id_div"},
    "FRAC": {"co_fracc": "id_div"},
    "BARRIO": {"barrios": "id_div"},
    "DPTO": {"comunas": "id_div"}
}
divs_path = pf.get_indic("divisiones")

In [46]:
paths_1

{'BARRIO': u'/Users/abenassi/github/tod/indicadores/barrios_censo_2010/barrios_censo_2010',
 'DPTO': u'/Users/abenassi/github/tod/indicadores/comunas_censo_2010/comunas_censo_2010',
 'FRAC': u'/Users/abenassi/github/tod/indicadores/fracciones_censo_2010/fracciones_censo_2010',
 'RADIO': u'/Users/abenassi/github/tod/indicadores/radios_censo_2010/radios_censo_2010'}

In [47]:
w = merge_shapefiles(paths_1, divs_path, group_fields_by_sf=group_fields_by_sf, 
                     prj_path=pf.get_prj("gkba_corregido"))

Merging 5 FRAC            fracciones_censo_2010
Merging 5 DPTO            comunas_censo_2010
Merging 5 RADIO           radios_censo_2010
Merging 5 BARRIO          barrios_censo_2010


## 2. Estaciones de subte, metrobús y ferrocarril.

In [48]:
paths_2 = {
    "est_sub_prem": pf.get_shp("estaciones-premetro"),
    "est_ffcc": pf.get_shp("estaciones-ferrocarril"),
    "est_subte": pf.get_shp("subte-estaciones"),
    "est_metrobus": pf.get_shp("metrobus-estaciones")
}
group_fields = {
    "estacion": ["estacion", "nombre"],
    "linea": ["linea", "metrobus", "lineasub"]
}
est_path = pf.get_indic("estaciones")

In [49]:
merge_shapefiles(paths_2, est_path, group_fields=group_fields, prj_path=pf.get_prj("gkba_corregido"))

Merging 1 est_metrobus    estaciones_de_metrobus
Merging 1 est_ffcc        estaciones_de_ferrocarril
Merging 1 est_sub_prem    estaciones_de_premetro
Merging 1 est_subte       estaciones_de_subte


## 3. Líneas de subte, metrobús, ferrocarril y colectivos.

In [50]:
paths_3 = {
    "lin_colectivos": pf.get_shp("recorridos_de_colectivos"),
    "lin_sub_prem": pf.get_shp("recorrido-premetro"),
    "lin_metrobus": pf.get_shp("recorrido_metrobus"),
    "lin_ffcc": pf.get_shp("linea-ferrocarril"),
    "lin_subte": pf.get_shp("subte-lineas")
}
group_fields = {
    "linea": ["linea", "metrobus", "lineasub"]
}
lines_path = pf.get_indic("lineas")

In [51]:
merge_shapefiles(paths_3, lines_path, group_fields=group_fields, prj_path=pf.get_prj("gkba_corregido"))

Merging 3 lin_ffcc        linea ferrocarril_1991
Merging 3 lin_metrobus    recorrido_de_metrobus
Merging 3 lin_subte       red_de_subte
Merging 3 lin_sub_prem    red_de_premetro
Merging 3 lin_colectivos  recorridos_de_colectivos


## 4. Buffers de las estaciones y sus indicadores.

In [52]:
replacements = {
    "accesibilidad_en_estaciones_de_subte": "est_sub_acc",
    "estaciones_de_ferrocarril": "est_ffcc",
    "estaciones_de_metrobus": "est_metrobus",
    "estaciones_de_premetro": "est_sub_prem",
    "estaciones_de_subte": "est_subte"
}
group_fields = {
    "estacion": ["estacion", "nombre"],
    "linea": ["linea", "metrobus"]
}
buffers_dir = pf.get("indicadores/buffers")
buffers_est_path = pf.get_indic("buffers_estaciones")

In [53]:
merge_shapefiles(buffers_dir, buffers_est_path, replacements=replacements, group_fields=group_fields,
                prj_path=pf.get_prj("gkba_corregido"))

Merging 5 est_subte-buffer750 estaciones_de_subte-buffer750
Merging 5 est_metrobus-buffer500 estaciones_de_metrobus-buffer500
Merging 5 est_metrobus-buffer750 estaciones_de_metrobus-buffer750
Merging 5 est_sub_prem-buffer500 estaciones_de_premetro-buffer500
Merging 5 est_sub_prem-buffer300 estaciones_de_premetro-buffer300
Merging 5 recorrido_de_metrobus-buffer2000 recorrido_de_metrobus-buffer2000
Merging 5 recorrido_de_metrobus-buffer1500 recorrido_de_metrobus-buffer1500
Merging 5 est_metrobus-buffer1500 estaciones_de_metrobus-buffer1500
Merging 5 est_subte-buffer500 estaciones_de_subte-buffer500
Merging 5 est_ffcc-buffer2000 estaciones_de_ferrocarril-buffer2000
Merging 5 est_sub_prem-buffer1000 estaciones_de_premetro-buffer1000
Merging 5 est_subte-buffer2000 estaciones_de_subte-buffer2000
Merging 5 est_metrobus-buffer300 estaciones_de_metrobus-buffer300
Merging 5 est_metrobus-buffer1000 estaciones_de_metrobus-buffer1000
Merging 5 est_ffcc-buffer1500 estaciones_de_ferrocarril-buffer150